<a href="https://colab.research.google.com/github/Eshan-K-Kaushal/Voice_Chatbot/blob/main/T5_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt


# from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
from pprint import pprint

import sys
import os
import glob

!pip install simplet5
from simplet5 import SimpleT5

In [3]:
PATH = '/content/Combined_Dataset_Test.csv'
dataset = pd.read_csv(PATH)
pd.set_option('display.max_colwidth', None)
#dataset = dataset[['Question', 'Answer']]
dataset = dataset.sample(frac=1, random_state=1).reset_index()
#print(dataset.head())

q_list = "question: " + dataset['Question']                          # questions list to feed the model
n_list = dataset['Answer'] + " </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
#print(df.head())

# print(df['source_text'][1])
# print('\n')
# print(df['target_text'][1])

# splitting data into train and test data
train_data, val_data = train_test_split(df[:77], test_size=0.2)
test_data = df[70:77]
print(train_data.shape, val_data.shape, test_data.shape)


(60, 2) (16, 2) (6, 2)


In [4]:
from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data,
            source_max_token_len=256,
            target_max_token_len=100,
            batch_size=4, max_epochs=10, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [5]:
model.load_model('/content/outputs/simplet5-epoch-9-train-loss-1.0054-val-loss-2.3018')

In [6]:
test_data['source_text']

70    question: I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step.\nAppend "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step.\nJasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?
71                                                                                           question: I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step. Append "Let's solve it step 

In [7]:
q_test = test_data['source_text'][71]
q_ans = test_data['target_text'][71]
q_ug = '''I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step.\nAppend "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step.\nAnn's favorite store was having a summer clearance. For $75 she bought 5 pairs of shorts for $7 each and 2 pairs of shoes for $10 each. She also bought 4 tops, all at the same price. How much did each top cost?'''
q_ug2 = '''I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step. Append "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step. Q: What's an example of a scientist in Britain who won the Physics Nobel Prize? '''
print('-'*50)
print("Question: ", q_ug, type(q_ug))
print('-'*50)
print("Question: ", q_test, type(q_test))
print('-'*50)
print("Answer: ",q_ans)

--------------------------------------------------
Question:  I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step.
Append "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step.
Ann's favorite store was having a summer clearance. For $75 she bought 5 pairs of shorts for $7 each and 2 pairs of shoes for $10 each. She also bought 4 tops, all at the same price. How much did each top cost? <class 'str'>
--------------------------------------------------
Question:  question: I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step. Append "Let's solve it step by step" after every question Q: <'Question a

In [8]:
#print(q_test)
print(q_ug2)
print('-'*50)
#predicted_ans = model.predict(q_test)[0]
predicted_ans_ug = model.predict(q_ug2)[0]
print(predicted_ans_ug)

I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step. Append "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step. Q: What's an example of a scientist in Britain who won the Physics Nobel Prize? 
--------------------------------------------------
Let's solve it step by step. The physicist George Clooney was born in Britain and won the Nobel Prize in Physics in 1975.


In [14]:
y_test = [a for a in test_data['target_text'] ]

# get the predicted answers
y_pred = []
for q in test_data['source_text']:
    y_pred.append(model.predict(q)[0])


In [ ]:
y_pred

In [ ]:
!pip install rouge_score

In [15]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [16]:
# get the rouge score between test and predicted data
precision_rouge_scores, recall_rouge_scores = [], []
for i in range(len(y_pred)):
    precision_rouge_scores.append(round(scorer.score(y_test[i], y_pred[i])['rouge1'][0],2))
    recall_rouge_scores.append(round(scorer.score(y_test[i], y_pred[i])['rouge1'][1],2))

In [17]:
print(f'The average precision rouge scores of q&a for test data is: {round(np.mean(precision_rouge_scores),2)}')
print(f'The average recall rouge scores of q&a for test data is: {round(np.mean(recall_rouge_scores),2)}')

The average precision rouge scores of q&a for test data is: 0.69
The average recall rouge scores of q&a for test data is: 0.31


In [18]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
pprint(scores)

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765),
 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}
